<a href="https://colab.research.google.com/github/Abhi23run/CSE8803_DLT_Project/blob/main/CodeBase/llama2_13b_chat_qlora_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U bitsandbytes
# !pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install transformers==4.31
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets
!pip install evaluate
!pip install -qqq trl==0.7.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 51.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.8 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.0
    Uninstalling tokenizers-0.15.0:
      Successfully uninstalled tokenizers-0.15.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 5.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 

In [2]:
%%capture
!pip install torch

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd /content/drive/MyDrive/CSE8803_DLT_Project/

/content/drive/MyDrive/CSE8803_DLT_Project


In [5]:
%ls

ashish_test/  finqa_dataset/          merged_model/  peft-dialogue-summary/
CodeBase/     llama2-docsum-adapter/  Papers/        README.md


In [9]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import transformers
from torch.optim import Adam
from tqdm import tqdm
from utils import *
import json
import pandas as pd
import torch
# from torch.utils.data import Dataset, DataLoader
import time
import evaluate
from datasets import Dataset, load_dataset
import random

In [7]:
dir_path = '/content/drive/MyDrive/finqa_dataset'

In [8]:
import os
os.chdir(dir_path)

In [10]:
with open(f"{dir_path}/train.json") as input_file:
        train_data = json.load(input_file)

with open(f"{dir_path}/dev.json") as input_file:
        valid_data = json.load(input_file)

with open(f"{dir_path}/test.json") as input_file:
        test_data = json.load(input_file)

In [11]:
assert len(train_data) == 6251, "The train data seems to be off, please check"
assert len(valid_data) == 883, "The validation data seems to be off, please check"
assert len(test_data) == 1147, "The test data seems to be off, please check"

In [19]:
train_data[0].keys()

dict_keys(['pre_text', 'post_text', 'filename', 'table_ori', 'table', 'qa', 'id', 'table_retrieved', 'text_retrieved', 'table_retrieved_all', 'text_retrieved_all'])

In [21]:
train_data[0]['id']

'ADI/2009/page_49.pdf-1'

In [35]:
def prepare_data(json_data, verbose=True):
    net=[]
    err_cnt=0
    for example in json_data:
        question = example["qa"]["question"]
        table = example["table"]
        id=example["id"]
        table_text = ""
        for row in table[1:]:
            this_sent = table_row_to_text(table[0], row)
            table_text += this_sent
        try:
            steps_text = format_steps(example["qa"]["steps"])
            inputs = {"id":id, "context": table_text, "question": question, "answer": steps_text}
            net.append(inputs)
        except:
            err_cnt+=1
            if verbose:
                print ("-"*25)
                print (example["filename"])
                print (example["qa"]["steps"])
                print ("-"*25+"\n")
    if err_cnt>0:
        print ("Net Errors:",err_cnt)
    return net

In [36]:
## Preparing huggingface dataset
data_splits = {'train': train_data, 'valid': valid_data, 'test': test_data}
datasets = {split: Dataset.from_pandas(pd.DataFrame(prepare_data(data, False))) for split, data in data_splits.items()}

Net Errors: 198
Net Errors: 35
Net Errors: 39


In [37]:
datasets

{'train': Dataset({
     features: ['id', 'context', 'question', 'answer'],
     num_rows: 6053
 }),
 'valid': Dataset({
     features: ['id', 'context', 'question', 'answer'],
     num_rows: 848
 }),
 'test': Dataset({
     features: ['id', 'context', 'question', 'answer'],
     num_rows: 1108
 })}

In [38]:
index=2
datasets['train'][index]

{'id': 'AAL/2018/page_13.pdf-2',
 'context': 'year the 2018 of gallons is 4447 ; the 2018 of average priceper gallon is $ 2.23 ; the 2018 of aircraft fuelexpense is $ 9896 ; the 2018 of percent of totaloperating expenses is 23.6% ( 23.6 % ) ;year the 2017 of gallons is 4352 ; the 2017 of average priceper gallon is 1.73 ; the 2017 of aircraft fuelexpense is 7510 ; the 2017 of percent of totaloperating expenses is 19.6% ( 19.6 % ) ;year the 2016 of gallons is 4347 ; the 2016 of average priceper gallon is 1.42 ; the 2016 of aircraft fuelexpense is 6180 ; the 2016 of percent of totaloperating expenses is 17.6% ( 17.6 % ) ;',
 'question': 'what was the total operating expenses in 2018 in millions',
 'answer': 'Step 1: Divide 9896 by 23.6%. This gives the result: 41932'}

**Loading in Llama7B-Chat mode**

In [22]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [27]:
import getpass

# Prompt for the Hugging Face token
hf_token = getpass.getpass("Enter your Hugging Face token: ")

# Use the token in your application
# For example, setting an environment variable (optional)
import os
os.environ['HUGGINGFACE_TOKEN'] = hf_token
# Now you can use Hugging Face APIs with the token
# Example: loading a model, accessing API, etc.


Enter your Hugging Face token: ··········


In [30]:
# model_id =  "NousResearch/Llama-2-7b-hf"
model_id = "meta-llama/Llama-2-13b-chat-hf"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto",use_auth_token=hf_token)

tokenizer = AutoTokenizer.from_pretrained(model_id,use_auth_token=hf_token)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [31]:
index=2
datasets['train'][index]

{'context': 'year the 2018 of gallons is 4447 ; the 2018 of average priceper gallon is $ 2.23 ; the 2018 of aircraft fuelexpense is $ 9896 ; the 2018 of percent of totaloperating expenses is 23.6% ( 23.6 % ) ;year the 2017 of gallons is 4352 ; the 2017 of average priceper gallon is 1.73 ; the 2017 of aircraft fuelexpense is 7510 ; the 2017 of percent of totaloperating expenses is 19.6% ( 19.6 % ) ;year the 2016 of gallons is 4347 ; the 2016 of average priceper gallon is 1.42 ; the 2016 of aircraft fuelexpense is 6180 ; the 2016 of percent of totaloperating expenses is 17.6% ( 17.6 % ) ;',
 'question': 'what was the total operating expenses in 2018 in millions',
 'answer': 'Step 1: Divide 9896 by 23.6%. This gives the result: 41932'}

In [32]:
def format_instruction(context: str, question: str, answer: str):
	return f"""### Instruction:
Answer the question using the information from the context below. Think step by step and solve the problem.

### Context:
{context.strip()}

### Question:
{question.strip()}

### Answer:
{answer}
""".strip()

In [39]:
def generate_instruction_dataset(data_point):

    return {
        "context": data_point["context"],
        "question": data_point["question"],
        "answer": data_point["answer"],
        "input_prompt": format_instruction(data_point["context"],data_point["question"],data_point["answer"])
    }

In [40]:
def process_dataset(data: Dataset):
    return (
        data.shuffle(seed=42)
        .map(generate_instruction_dataset)
    )

In [41]:
## APPLYING PREPROCESSING ON WHOLE DATASET
datasets["train"] = process_dataset(datasets["train"])
datasets["test"] = process_dataset(datasets["test"])
datasets["valid"] = process_dataset(datasets["valid"])

Map:   0%|          | 0/6053 [00:00<?, ? examples/s]

Map:   0%|          | 0/1108 [00:00<?, ? examples/s]

Map:   0%|          | 0/848 [00:00<?, ? examples/s]

In [42]:
index=0
print(datasets["train"]["input_prompt"][index])

### Instruction:
Answer the question using the information from the context below. Think step by step and solve the problem.

### Context:
reporting unit the retail brokerage of december 31 2012 goodwill is $ 1791.8 ; the retail brokerage of december 31 2012 % ( % ) of fair value to book value is 190% ( 190 % ) ;reporting unit the market making of december 31 2012 goodwill is 142.4 ; the market making of december 31 2012 % ( % ) of fair value to book value is 115% ( 115 % ) ;reporting unit the total goodwill of december 31 2012 goodwill is $ 1934.2 ; the total goodwill of december 31 2012 % ( % ) of fair value to book value is ;

### Question:
what percentage of total goodwill is comprised of market making at december 31 2012?

### Answer:
Step 1: Divide 142.4 by 1934.2. This gives the result: 7%


In [48]:
import warnings
warnings.filterwarnings('ignore')
# index = 4
index = 0

context = datasets['train'][index]['context']
answer = datasets['train'][index]['answer']
question = datasets['train'][index]['question']

prompt = f"""
Answer the question using the information from the context below. Think step by step and solve the problem.

### Context:
{context}

### Question:
{question}

### Answer:
"""

inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        temperature=0.2,
        do_sample=True,
        max_new_tokens=2048,
    )[0],
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE ANSWER:\n{answer}\n')
print(dash_line)
print(f'MODEL GENERATED ANSWER - ZERO SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Answer the question using the information from the context below. Think step by step and solve the problem.

### Context:
reporting unit the retail brokerage of december 31 2012 goodwill is $ 1791.8 ; the retail brokerage of december 31 2012 % ( % ) of fair value to book value is 190% ( 190 % ) ;reporting unit the market making of december 31 2012 goodwill is 142.4 ; the market making of december 31 2012 % ( % ) of fair value to book value is 115% ( 115 % ) ;reporting unit the total goodwill of december 31 2012 goodwill is $ 1934.2 ; the total goodwill of december 31 2012 % ( % ) of fair value to book value is ;

### Question:
what percentage of total goodwill is comprised of market making at december 31 2012?

### Answer:

---------------------------------------------------------------------------------------------------
BASELINE ANSWER:
Step 1: Divide 142.4 by 1934.2. Th

In [47]:
answer

'Step 1: Divide 142.4 by 1934.2. This gives the result: 7%'